In [42]:
from nltk.corpus import wordnet as wn
for synset in wn.synsets('subspecies'):
    print(synset.lemma_names())

['subspecies', 'race']


In [31]:
wn.synsets('race')

[Synset('race.n.01'),
 Synset('race.n.02'),
 Synset('race.n.03'),
 Synset('subspecies.n.01'),
 Synset('slipstream.n.01'),
 Synset('raceway.n.01'),
 Synset('rush.v.01'),
 Synset('race.v.02'),
 Synset('race.v.03'),
 Synset('race.v.04')]

In [40]:
motorcar = wn.synset('black_race.n.01')
types_of_motorcar = motorcar.hypernyms()
types_of_motorcar

[Synset('race.n.03')]

In [38]:
motorcar = wn.synset('race.n.03')
types_of_motorcar = motorcar.hyponyms()
types_of_motorcar

[Synset('black_race.n.01'),
 Synset('color.n.04'),
 Synset('indian_race.n.01'),
 Synset('indian_race.n.02'),
 Synset('master_race.n.01'),
 Synset('slavic_people.n.01'),
 Synset('white_race.n.01'),
 Synset('yellow_race.n.01')]

In [39]:
right = wn.synset('ethnology.n.01') 
minke = wn.synset('race.n.03')
# 「露脊鯨」與「小鬚鯨」在上位詞組中最低位的詞組
right.lowest_common_hypernyms(minke)
right.path_similarity(minke)

0.08333333333333333

In [2]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-roberta-large-v1')

In [7]:
taste = ['FLAVOUR'.lower()]
category = ['Sensation']

embeddings1 = model.encode(taste, convert_to_tensor=True)
embeddings2 = model.encode(category, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(taste)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(taste[i], category[i], cosine_scores[i][i]))

flavour 		 Sensation 		 Score: 0.4958


In [44]:
import json

In [ ]:
with open('../data/wiki/wiki_word_id2sents.json') as f:
    data = json.loads(f.read())

In [ ]:
with open('../data/remap.word_id.topics.examples.json') as f:
    word_id2topics = json.loads(f.read())

In [ ]:
for word in data:
    if word.startswith('slug.noun'):
        if word in word_id2topics:
            print(word)
            print(word_id2topics[word]['topics'])
            print(data[word])

In [ ]:
for word in data:
    if word.startswith('bank.noun'):
        print(word)

In [ ]:
with open('../data/wiki/wiki_href2def.json') as f:
    wiki = json.loads(f.read())

In [ ]:
for word in wiki:
    if '(geology)' in word:
        print(word)

In [ ]:
sent = 'The banks of Switzerland and the insurance companies in Switzerland together produce eleven per cent of the gross domestic product.'
for word in wiki:
    if 'bank' in wiki[word][0]:
        print(word)

In [45]:
with open('../data/jsonl_file/0.0_top3_brt_map_cam.jsonl') as f:
    brt = [json.loads(line) for line in f.readlines()]

In [ ]:
with open('../data/words2defs.json') as f:
    word2def = json.loads(f.read())

In [ ]:
corpus_sentences = word2def['taste'] + category['Sensation']

In [ ]:
category = defaultdict(list)
for data in brt:
    if data['brt_word'] != 'taste':
        category[data['category']].append(data['en_def'])

In [ ]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import time

model = SentenceTransformer('all-roberta-large-v1')
corpus_embeddings = model.encode(corpus_sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)
print("Start clustering")
start_time = time.time()

clusters = util.community_detection(corpus_embeddings, min_community_size=1, threshold=0.2)

print("Clustering done after {:.2f} sec".format(time.time() - start_time))

#Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", corpus_sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", corpus_sentences[sentence_id])

---

In [ ]:
import json

with open('../data/merged_brt_group.json') as f:
    merged_brt = json.loads(f.read())

In [ ]:
name = {}
for k, v in merged_brt.items():
    for idx, group in enumerate(v):
        for g in group:
            name[g] = k + '_' + str(idx)

In [ ]:
# with open('../data/original_new.json', 'w') as f:
#     f.write(json.dumps(name))

In [ ]:
with open('../data/word_id.topics.examples.json') as f:
    original = json.loads(f.read())

In [ ]:
new_dict = {}
for key, value in original.items():
    temp = []
    for group in value['topics']:
        topic = group.split(' ', 1)[1]
        temp.append(name[topic])
    new_dict[key] = {'topics':list(set(temp)),
                    'examples': value['examples'],
                    'pos': value['pos'],
                    'headword': value['headword']}

In [ ]:
with open('../data/merged_word_id.topics.json', 'w') as f:
    f.write(json.dumps(new_dict))